In [1]:
import re, subprocess, pyodbc , json
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import mongoConn, connstr  
import pandas as pd
from faker import Faker
fake = Faker()

In [2]:
engine = create_engine(connstr)
conn = engine.connect()
query = """ \
SELECT QPK, QGROUP, IdText, GroupName, QuestionTypeCode, QuestionTypeID, QTEXT    
FROM  vw_MetricsCompositeKeys
WHERE PK_Form LIKE '%2022-Q1-CIO%'
AND IdText LIKE '4.1._[a-z]'
"""  
df = pd.read_sql(query,con=conn) 
conn.close()
df = df.loc[df.IdText != '']
df

,QPK,QGROUP,IdText,GroupName,QuestionTypeCode,QuestionTypeID,QTEXT
0,27093,3207,4.1.1a,S4,CNT,2,Critical software incorporating security measure
1,27094,3207,4.1.1b,S4,CNT,2,Critical software for which risk of not incorp...
2,27095,3207,4.1.1c,S4,CNT,2,Critical software where security measure is no...
3,27097,3207,4.1.2a,S4,CNT,2,Critical software incorporating security measure
4,27098,3207,4.1.2b,S4,CNT,2,Critical software for which risk of not incorp...
5,27099,3207,4.1.2c,S4,CNT,2,Critical software where security measure is no...
6,27101,3207,4.1.3a,S4,CNT,2,Critical software incorporating security measure
7,27102,3207,4.1.3b,S4,CNT,2,Critical software for which risk of not incorp...
8,27103,3207,4.1.3c,S4,CNT,2,Critical software where security measure is no...
9,27105,3207,4.1.4a,S4,CNT,2,Critical software incorporating security measure


In [3]:
def idtxt(r):
    tmp = """ DECLARE @ANS_$1 INT = (SELECT CASE @PK_QuestionGroup WHEN 0 THEN TableAns ELSE COALESCE(CONVERT(INT,MetricAns),0) END FROM @ListofQuestions2 WHERE identifier_text='$2')"""
    tmp=tmp.replace('$1',r['IdText'].replace('.','_'))
    tmp=tmp.replace('$2',r['IdText'])
    return tmp

In [7]:
for i,r in df.iterrows(): 
    p = idtxt(r)
    print( p ) 

 DECLARE @ANS_4_1_1a INT = (SELECT CASE @PK_QuestionGroup WHEN 0 THEN TableAns ELSE COALESCE(CONVERT(INT,MetricAns),0) END FROM @ListofQuestions2 WHERE identifier_text='4.1.1a')
 DECLARE @ANS_4_1_1b INT = (SELECT CASE @PK_QuestionGroup WHEN 0 THEN TableAns ELSE COALESCE(CONVERT(INT,MetricAns),0) END FROM @ListofQuestions2 WHERE identifier_text='4.1.1b')
 DECLARE @ANS_4_1_1c INT = (SELECT CASE @PK_QuestionGroup WHEN 0 THEN TableAns ELSE COALESCE(CONVERT(INT,MetricAns),0) END FROM @ListofQuestions2 WHERE identifier_text='4.1.1c')
 DECLARE @ANS_4_1_2a INT = (SELECT CASE @PK_QuestionGroup WHEN 0 THEN TableAns ELSE COALESCE(CONVERT(INT,MetricAns),0) END FROM @ListofQuestions2 WHERE identifier_text='4.1.2a')
 DECLARE @ANS_4_1_2b INT = (SELECT CASE @PK_QuestionGroup WHEN 0 THEN TableAns ELSE COALESCE(CONVERT(INT,MetricAns),0) END FROM @ListofQuestions2 WHERE identifier_text='4.1.2b')
 DECLARE @ANS_4_1_2c INT = (SELECT CASE @PK_QuestionGroup WHEN 0 THEN TableAns ELSE COALESCE(CONVERT(INT,Metri

In [17]:
 for i,r in df.iterrows(): 
    p = r['IdText'].replace('.','_')
    idd = r['IdText']
    print( 
    f"""
		IF @ANS_{p} > @ANS_4_1
		BEGIN
			INSERT INTO @ErrorTable  
			SELECT  PK_Question, PK_FormPage, page_sort_pos, questgroup_sort_pos, quest_sort_pos
			, ('The number provided for ' + identifier_text + ' cannot exceed the value of 4.1 : '  + QuestionText) Error, identifier_text
			FROM vw_OrgSubQuestions WHERE  identifier_text='{idd}' AND PK_OrgSubmission = @PK_OrgSubmission  
		END"""
    )
    


		IF @ANS_4_1_1a > @ANS_4_1
		BEGIN
			INSERT INTO @ErrorTable  
			SELECT  PK_Question, PK_FormPage, page_sort_pos, questgroup_sort_pos, quest_sort_pos
			, ('The number provided for ' + identifier_text + ' cannot exceed the value of 4.1 : '  + QuestionText) Error, identifier_text
			FROM vw_OrgSubQuestions WHERE  identifier_text='4.1.1a' AND PK_OrgSubmission = @PK_OrgSubmission  
		END

		IF @ANS_4_1_1b > @ANS_4_1
		BEGIN
			INSERT INTO @ErrorTable  
			SELECT  PK_Question, PK_FormPage, page_sort_pos, questgroup_sort_pos, quest_sort_pos
			, ('The number provided for ' + identifier_text + ' cannot exceed the value of 4.1 : '  + QuestionText) Error, identifier_text
			FROM vw_OrgSubQuestions WHERE  identifier_text='4.1.1b' AND PK_OrgSubmission = @PK_OrgSubmission  
		END

		IF @ANS_4_1_1c > @ANS_4_1
		BEGIN
			INSERT INTO @ErrorTable  
			SELECT  PK_Question, PK_FormPage, page_sort_pos, questgroup_sort_pos, quest_sort_pos
			, ('The number provided for ' + identifier_text + ' cannot 